In [1]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 129kB/s 
     |████████████████████████████████| 421.8MB 26kB/s 
     |████████████████████████████████| 6.7MB 47.2MB/s 
     |████████████████████████████████| 983kB 15.8MB/s 
     |████████████████████████████████| 245kB 50.9MB/s 
     |████████████████████████████████| 675kB 48.8MB/s 
     |████████████████████████████████| 471kB 41.9MB/s 
     |████████████████████████████████| 3.9MB 48.6MB/s 
     |████████████████████████████████| 450kB 50.8MB/s 
     |████████████████████████████████| 890kB 53.1MB/s 
     |████████████████████████████████| 3.8MB 42.9MB/s 
     |████████████████████████████████| 1.1MB 52.4MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [36]:
train_b=pd.read_csv("/content/drive/My Drive/minor/german_dataset/german_dataset.tsv",sep='\t',encoding='utf-8') #training data
test_b=pd.read_csv("/content/drive/My Drive/minor/german_dataset/hasoc_de_test_gold.tsv",sep='\t',encoding='utf-8') #testing data
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_1'],test_size=0.15,random_state=42) #training and validation split
target_names=['HOF','NOT'] #names of classes

In [8]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [9]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-german-cased' #bert predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for bert
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get bert classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) #get trainable learner for bert

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')
/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:520: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


preprocessing train...
language: de
train sequence lengths:
	mean : 22
	95percentile : 42
	99percentile : 47


Is Multi-Label? False
preprocessing test...
language: de
test sequence lengths:
	mean : 23
	95percentile : 42
	99percentile : 45


In [10]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 541 steps, validate for 18 steps
Epoch 1/4
541/541 [==============================] - 265s 490ms/step - loss: 0.3450 - accuracy: 0.8857 - val_loss: 0.3151 - val_accuracy: 0.8988
Epoch 2/4
541/541 [==============================] - 250s 462ms/step - loss: 0.2950 - accuracy: 0.8934 - val_loss: 0.2701 - val_accuracy: 0.9023
Epoch 3/4
541/541 [==============================] - 250s 462ms/step - loss: 0.2047 - accuracy: 0.9211 - val_loss: 0.3111 - val_accuracy: 0.8953
Epoch 4/4
541/541 [==============================] - 250s 462ms/step - loss: 0.0613 - accuracy: 0.9809 - val_loss: 0.3967 - val_accuracy: 0.8918


In [11]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.43      0.21      0.28        58
           1       0.92      0.97      0.94       515

    accuracy                           0.89       573
   macro avg       0.67      0.59      0.61       573
weighted avg       0.87      0.89      0.87       573



array([[ 12,  46],
       [ 16, 499]])

In [12]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  bert

In [33]:
predictor.get_classes()

['HOF', 'NOT']

In [37]:
test_b_text=list(test_b['text']) #get text for testing
test_b['task_1']=test_b['task_1'].map({'HOF':0,'NOT':1})
test_b_label=list(test_b['task_1']) #get label of testing
# val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data

In [38]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [39]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [40]:
from sklearn.metrics import classification_report

In [41]:
print(test_b_label)

[0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [42]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.51      0.13      0.21       136
           1       0.86      0.98      0.91       714

    accuracy                           0.84       850
   macro avg       0.68      0.55      0.56       850
weighted avg       0.80      0.84      0.80       850

